In [12]:
import json
import openai

In [13]:
client = openai.Client()

In [14]:
def saudacao_por_periodo(hora):
    if 5 <= 5 < 12:
        return json.dumps({"saudacao": "Bom dia!"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao": "Boa tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao": "Boa noite!"})
    else:
        return json.dumps({"saudacao": "Boa madrugada!"})

In [16]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_periodo",
            "description": "Retorna uma saudacao",
            "parameters": {
                "type": "object",
                "properties": {
                    "hora": {
                        "type": "integer",
                        "description": "A hora do dia em formato de 24 horas"
                    },
                },
                "required": ["hora"]
            }
        }
    }
]

In [17]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo
}

In [25]:
mensagens = [{"role": "user", "content": "Qual saudacao me dar se o modelo for 23h?"}]

In [27]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo-0125",
    tools=tools,
    tool_choice="auto"
)

In [28]:
mensagem_resposta = resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1tba0QlCo9lifMKOQHr8wKDb', function=Function(arguments='{"hora":23}', name='saudacao_por_periodo'), type='function')])

In [29]:
tool_calls = mensagem_resposta.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_1tba0QlCo9lifMKOQHr8wKDb', function=Function(arguments='{"hora":23}', name='saudacao_por_periodo'), type='function')]

In [30]:
if tool_calls:
    mensagens.append(mensagem_resposta)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )
        
        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens
    )

In [31]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='Boa noite!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)